In [17]:
import os
import json
import random
import pickle
import tarfile
import requests
import pandas as pd
from tqdm import tqdm
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_relevancy
from groq import Groq
import time

In [43]:
pwd

'/Users/victorgmoreno/repositorios/IA024_NLP_PROJECTS/atividade_11'

In [14]:
GROQ_API_KEY = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [44]:
import os
import json
import pickle
import random
import time
from tqdm import tqdm
import pandas as pd

def load_data():
    data_directory = 'data'
    articles_file = os.path.join(data_directory, "context_articles.json")
    questions_file = os.path.join(data_directory, "test_questions.json")

    # Carregar artigos de contexto e conjunto de teste
    if os.path.exists(articles_file) and os.path.exists(questions_file):
        with open(articles_file, 'r') as f:
            context_articles = json.load(f)
        with open(questions_file, 'r') as f:
            test_set = json.load(f)
        return context_articles, test_set
    else:
        print("Files not found, check your dataset path.")
        return None, None

def process_and_evaluate(context_articles, test_set):
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    # Garantindo que o diretório de dados exista
    os.makedirs(data_directory, exist_ok=True)

    # Carregando ou inicializando o dicionário de QA
    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            qa = pickle.load(f)
    else:
        qa = {
            'question': [],
            'ground_truth': [],
            'contexts': [],
            'answer': [],
            'faithfulness': [],
            'answer_relevancy': [],
            'context_relevancy': []
        }

    # Assumindo que 'test_set' é uma lista de dicionários
    with tqdm(total=len(test_set)) as pbar:
        for entry in test_set:
            if entry['question'] not in qa['question']:
                try:
                    # Simulação de avaliação
                    result = {'faithfulness': random.random(), 'answer_relevancy': random.random(), 'context_relevancy': random.random()}
                    
                    qa['question'].append(entry['question'])
                    qa['ground_truth'].append(entry['ground_truth'])
                    qa['contexts'].append(context_articles)  # Exemplo simplificado
                    qa['answer'].append(entry['answer'])
                    for key in result:
                        qa[key].append(result[key])

                    if len(qa['question']) % 5 == 0:
                        with open(save_file, 'wb') as f:
                            pickle.dump(qa, f)
                except Exception as e:
                    print(f"Error at index {len(qa['question'])}: {e}")
                    time.sleep(120)
                pbar.update(1)

        # Salvando os dados finais
        with open(save_file, 'wb') as f:
            pickle.dump(qa, f)

def load_and_display_results():
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            data = pickle.load(f)
        df = pd.DataFrame(data)
        print(df)
    else:
        print(f"Arquivo {save_file} não encontrado.")

if __name__ == "__main__":
    context_articles, test_set = load_data()
    if context_articles and test_set:
        process_and_evaluate(context_articles, test_set)
        load_and_display_results()


  0%|          | 0/50 [00:00<?, ?it/s]

Error at index 2: 'ground_truth'


  2%|▏         | 1/50 [02:00<1:38:00, 120.00s/it]

Error at index 3: 'ground_truth'


In [15]:
def setup_dataset():
    DATA_DIR = 'data'
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)

    context_articles = "https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz"
    filename = 'context_articles.tar.gz'
    filepath = os.path.join(DATA_DIR, filename)

    if not os.path.exists(filepath):
        print(f"Downloading {filename}...")
        response = requests.get(context_articles)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        print(f"{filename} downloaded.")
        
        with tarfile.open(filepath, 'r:gz') as tar:
            tar.extractall(path=DATA_DIR)
        print(f"{filename} extracted.")


In [37]:
import os
import pickle
import random
import time
from tqdm import tqdm

def process_and_evaluate():
    # Definindo o diretório e o arquivo de dados
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    # Garantindo que o diretório de dados exista
    os.makedirs(data_directory, exist_ok=True)

    # Carregando ou inicializando o dicionário de QA
    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            qa = pickle.load(f)
    else:
        qa = {
            'question': [],
            'ground_truth': [],
            'contexts': [],
            'answer': [],
            'faithfulness': [],
            'answer_relevancy': [],
            'context_relevancy': []
        }

    # Simulando dados de entrada para processamento
    dataset = [
        {'question': "What is AI?", 'ground_truth': "Study of intelligent agents.", 'contexts': "AI is used in various fields.", 'answer': "AI stands for Artificial Intelligence."}
    ]

    with tqdm(total=len(dataset)) as pbar:
        for entry in dataset:
            if entry['question'] not in qa['question']:
                try:
                    # Simulação de avaliação
                    result = {'faithfulness': random.random(), 'answer_relevancy': random.random(), 'context_relevancy': random.random()}
                    
                    qa['question'].append(entry['question'])
                    qa['ground_truth'].append(entry['ground_truth'])
                    qa['contexts'].append(entry['contexts'])
                    qa['answer'].append(entry['answer'])
                    for key in result:
                        qa[key].append(result[key])

                    if len(qa['question']) % 5 == 0:
                        with open(save_file, 'wb') as f:
                            pickle.dump(qa, f)
                except Exception as e:
                    print(f"Error at index {len(qa['question'])}: {e}")
                    time.sleep(120)
                pbar.update(1)

        with open(save_file, 'wb') as f:
            pickle.dump(qa, f)


In [38]:
def load_and_display_results():
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            data = pickle.load(f)
        df = pd.DataFrame(data)
        print(df)
    else:
        print(f"Arquivo {save_file} não encontrado.")

In [39]:
if __name__ == "__main__":
    setup_dataset()
    process_and_evaluate()
    # Carregar e exibir os resultados
    with open('qa_data.pickle', 'rb') as f:
        data = pickle.load(f)
    df = pd.DataFrame(data)
    print(df.describe())
    df.hist(figsize=(10,8), layout=(1,3))


  0%|          | 0/1 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'qa_data.pickle'

In [40]:
#Adaptado do Ramon Simoes
save_file = "qa_data.pickle"

if os.path.exists(save_file):
    with open(save_file, 'rb') as f:
        qa = pickle.load(f)
else:
    qa = {'question': [], 'ground_truth': [], 'contexts': [], 'answer': [],
          'faithfulness': [], 'answer_relevancy': [], 'context_relevancy': []
         }

last_processed_index = 0
with tqdm(total=len(dataset)) as pbar:
    while last_processed_index < len(dataset):

        q = Dataset.from_dict(dataset[last_processed_index : last_processed_index + 1 ])

        if q['question'] in qa['question']:
            last_processed_index += 1
            pbar.update(1)
            continue

        try:
            qa['question'].append(q['question'])
            qa['ground_truth'].append(q['ground_truth'])
            qa['contexts'].append(q['contexts'])
            qa['answer'].append(q['answer'])

            result = evaluate(q, metrics, llm=llm, embeddings=embedder)

            for r in result:
                qa[r].append(result[r])

            if len(qa['question']) % 5 == 0:
                with open(save_file, 'wb') as f:
                    pickle.dump(qa, f)

                sleep_time = random.uniform(5, 20)
                time.sleep(sleep_time)

            last_processed_index += 1
            pbar.update(1)

        except Exception as e:
            print(f"\nErro no indice {last_processed_index}: {e}")
            print("Rate limit excedido... esperando 120s")
            time.sleep(120)

with open(save_file, 'wb') as f:
    pickle.dump(qa, f)

NameError: name 'dataset' is not defined